# Sudoku-Python-ORTools-Z3 : OR-Tools et Z3 (Python)

**Navigation** : [<< Python Backtracking](Sudoku-Python-Backtracking.ipynb) | [Index](README.md) | [Python Genetic >>](Sudoku-Python-Genetic.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Modeliser le Sudoku avec des solveurs declaratifs (OR-Tools CP-SAT et Z3 SMT)
2. Comprendre la difference entre programmation par contraintes et SMT
3. Comparer les performances des solveurs industriels avec le backtracking
4. Utiliser des BitVectors dans Z3 pour optimiser les representations

**Duree estimee** : ~10 min | **Prerequis** : [Sudoku-0 Environment](Sudoku-0-Environment.ipynb) | **Lien** : Voir [Search-8 CSP Advanced](../Search/Foundations/Search-8-CSP-Advanced.ipynb)

---

Ce notebook implemente des solveurs de Sudoku utilisant des approches **declaratives** (par opposition a l'approche imperative du backtracking):

- **Google OR-Tools CP-SAT**: Solveur de Programmation par Contraintes avec techniques SAT
- **Microsoft Z3**: Solveur SMT (Satisfiability Modulo Theories)

Ces approches sont equivalentes aux notebooks C# `Sudoku-3-ORTools.ipynb` et `Sudoku-4-Z3.ipynb`.

## Paradigme declaratif vs imperatif

| Aspect | Backtracking (imperatif) | CP-SAT/SMT (declaratif) |
|--------|--------------------------|-------------------------|
| **Comment resoudre** | On programme l'algorithme | On decrit les contraintes |
| **Optimisations** | Manuelles (MRV, etc.) | Automatiques par le solveur |
| **Expressivite** | Code specifique au probleme | Modele generique reutilisable |
| **Maintenance** | Algorithme a maintenir | Seulement les contraintes |

## Avantages des solveurs industriels

1. **Annees d'optimisation**: CP-SAT et Z3 integrent des decennies de recherche en optimisation
2. **Propagation de contraintes**: Reduction automatique des domaines
3. **Apprentissage de conflits**: Evite de repeter les memes erreurs (CDCL)
4. **Parallelisation**: Exploitation automatique du multi-core

## Installation

```bash
pip install ortools z3-solver
```

In [ ]:
# Imports
import time
from typing import List, Optional, Tuple

# Vérifier les installations
try:
    from ortools.sat.python import cp_model
    print(f"OR-Tools importé avec succès")
except ImportError:
    print("OR-Tools non installé. Exécutez: pip install ortools")

try:
    from z3 import *
    print(f"Z3 importé avec succès")
except ImportError:
    print("Z3 non installé. Exécutez: pip install z3-solver")

## 1. Classe SudokuGrid (réutilisée)

Même représentation que dans le notebook Backtracking. La classe est indépendante du solveur utilisé, ce qui permet de comparer facilement différentes approches sur les mêmes puzzles.

In [ ]:
class SudokuGrid:
    """Représentation d'une grille de Sudoku 9x9."""
    
    def __init__(self, grid: Optional[List[List[int]]] = None):
        if grid is None:
            self.cells = [[0] * 9 for _ in range(9)]
        else:
            self.cells = [row[:] for row in grid]
    
    @classmethod
    def from_string(cls, s: str) -> 'SudokuGrid':
        s = s.replace('.', '0').replace(' ', '').replace('\n', '')
        if len(s) != 81:
            raise ValueError(f"La chaîne doit avoir 81 caractères")
        grid = cls()
        for i in range(81):
            grid.cells[i // 9][i % 9] = int(s[i])
        return grid
    
    def clone(self) -> 'SudokuGrid':
        return SudokuGrid(self.cells)
    
    def __str__(self) -> str:
        lines = []
        for r in range(9):
            if r > 0 and r % 3 == 0:
                lines.append('-' * 21)
            row_str = ''
            for c in range(9):
                if c > 0 and c % 3 == 0:
                    row_str += '| '
                val = self.cells[r][c]
                row_str += (str(val) if val != 0 else '.') + ' '
            lines.append(row_str)
        return '\n'.join(lines)

def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

# Charger puzzles
easy_puzzles = load_puzzles('Puzzles/Sudoku_Easy51.txt', max_puzzles=10)
hard_puzzles = load_puzzles('Puzzles/Sudoku_hardest.txt')
print(f"Puzzles chargés: {len(easy_puzzles)} faciles, {len(hard_puzzles)} difficiles")

## 2. Solveur OR-Tools CP-SAT

**Google OR-Tools** est une suite d'optimisation open-source. Le solveur **CP-SAT** (Constraint Programming with SAT) combine:

- **Programmation par contraintes (CP)**: Modélisation naturelle avec domaines et contraintes
- **Techniques SAT**: Apprentissage de clauses, propagation unitaire, backjumping

### Modélisation du Sudoku en CP

Le modèle se construit en 3 étapes:

1. **Variables**: 81 variables entières, domaine {1..9}
   ```python
   cells[(i, j)] = model.NewIntVar(1, 9, f'cell_{i}_{j}')
   ```

2. **Contraintes de valeurs initiales**: Fixer les cases connues
   ```python
   cells[(i, j)] = model.NewConstant(valeur_initiale)
   ```

3. **Contraintes AllDifferent**: La contrainte clé du Sudoku
   ```python
   model.AddAllDifferent([cells[(i, j)] for j in range(9)])  # Ligne
   model.AddAllDifferent([cells[(i, j)] for i in range(9)])  # Colonne
   model.AddAllDifferent([...])  # Bloc 3x3
   ```

### Pourquoi CP-SAT est si rapide?

- **Propagation AllDifferent**: Algorithme spécialisé O(n sqrt(n)) qui détecte les inconsistances
- **Arc Consistency**: Réduit les domaines avant même de commencer la recherche
- **CDCL**: Apprend des conflits pour éviter de les répéter
- **Restarts**: Stratégie de redémarrage pour éviter les chemins sans issue

In [ ]:
class ORToolsSolver:
    """Solveur Sudoku utilisant OR-Tools CP-SAT."""
    
    def solve(self, grid: SudokuGrid) -> bool:
        """Résout le Sudoku avec OR-Tools CP-SAT."""
        model = cp_model.CpModel()
        
        # Créer les variables: cells[i][j] in {1..9}
        cells = {}
        for i in range(9):
            for j in range(9):
                if grid.cells[i][j] != 0:
                    # Valeur fixée
                    cells[(i, j)] = model.NewConstant(grid.cells[i][j])
                else:
                    # Variable libre
                    cells[(i, j)] = model.NewIntVar(1, 9, f'cell_{i}_{j}')
        
        # Contraintes: toutes différentes par ligne
        for i in range(9):
            model.AddAllDifferent([cells[(i, j)] for j in range(9)])
        
        # Contraintes: toutes différentes par colonne
        for j in range(9):
            model.AddAllDifferent([cells[(i, j)] for i in range(9)])
        
        # Contraintes: toutes différentes par bloc 3x3
        for box_row in range(3):
            for box_col in range(3):
                box_cells = []
                for i in range(3):
                    for j in range(3):
                        box_cells.append(cells[(box_row * 3 + i, box_col * 3 + j)])
                model.AddAllDifferent(box_cells)
        
        # Résoudre
        solver = cp_model.CpSolver()
        status = solver.Solve(model)
        
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # Extraire la solution
            for i in range(9):
                for j in range(9):
                    grid.cells[i][j] = solver.Value(cells[(i, j)])
            return True
        
        return False

# Test
ortools_solver = ORToolsSolver()
test_grid = SudokuGrid.from_string(hard_puzzles[0])
print("Puzzle difficile:")
print(test_grid)

start = time.time()
solved = ortools_solver.solve(test_grid)
elapsed = (time.time() - start) * 1000

print(f"\nRésolu: {solved} en {elapsed:.2f} ms")
print("\nSolution:")
print(test_grid)

## 3. Solveur Z3 SMT

**Microsoft Z3** est un solveur **SMT** (Satisfiability Modulo Theories) - il combine la puissance des solveurs SAT avec des théories mathématiques (arithmétique, tableaux, bitvectors, etc.).

### Différence entre SAT et SMT

| SAT | SMT |
|-----|-----|
| Variables booléennes uniquement | Variables typées (Int, Real, BitVec, Array...) |
| Clauses propositionnelles | Formules de premier ordre |
| `(x OR y) AND (NOT x OR z)` | `x + y > 10 AND x < 5` |

### Modélisation du Sudoku en SMT

1. **Variables entières**: `X[i][j]` est un entier symbolique
   ```python
   X = [[Int(f'x_{i}_{j}') for j in range(9)] for i in range(9)]
   ```

2. **Contraintes de domaine**: Exprimer que chaque variable est dans {1..9}
   ```python
   solver.add(X[i][j] >= 1, X[i][j] <= 9)
   ```

3. **Contraintes Distinct**: Équivalent de AllDifferent
   ```python
   solver.add(Distinct([X[i][j] for j in range(9)]))
   ```

### Quand utiliser Z3 plutôt que CP-SAT?

- **Z3**: Plus flexible, supporte des contraintes complexes (non-linéaire, quantificateurs)
- **CP-SAT**: Plus rapide sur les problèmes combinatoires classiques comme le Sudoku
- **Les deux**: Excellent pour prototyper des modèles de contraintes

In [ ]:
class Z3Solver:
    """Solveur Sudoku utilisant Z3 SMT."""
    
    def solve(self, grid: SudokuGrid) -> bool:
        """Résout le Sudoku avec Z3."""
        # Créer les variables: X[i][j] est un entier 1-9
        X = [[Int(f'x_{i}_{j}') for j in range(9)] for i in range(9)]
        
        solver = Solver()
        
        # Contraintes de domaine: 1 <= X[i][j] <= 9
        for i in range(9):
            for j in range(9):
                solver.add(X[i][j] >= 1, X[i][j] <= 9)
        
        # Valeurs initiales
        for i in range(9):
            for j in range(9):
                if grid.cells[i][j] != 0:
                    solver.add(X[i][j] == grid.cells[i][j])
        
        # Contraintes: toutes différentes par ligne
        for i in range(9):
            solver.add(Distinct([X[i][j] for j in range(9)]))
        
        # Contraintes: toutes différentes par colonne
        for j in range(9):
            solver.add(Distinct([X[i][j] for i in range(9)]))
        
        # Contraintes: toutes différentes par bloc 3x3
        for box_row in range(3):
            for box_col in range(3):
                box_cells = []
                for i in range(3):
                    for j in range(3):
                        box_cells.append(X[box_row * 3 + i][box_col * 3 + j])
                solver.add(Distinct(box_cells))
        
        # Résoudre
        if solver.check() == sat:
            model = solver.model()
            for i in range(9):
                for j in range(9):
                    grid.cells[i][j] = model.evaluate(X[i][j]).as_long()
            return True
        
        return False

# Test
z3_solver = Z3Solver()
test_grid = SudokuGrid.from_string(hard_puzzles[0])
print("Puzzle difficile:")
print(test_grid)

start = time.time()
solved = z3_solver.solve(test_grid)
elapsed = (time.time() - start) * 1000

print(f"\nRésolu: {solved} en {elapsed:.2f} ms")
print("\nSolution:")
print(test_grid)

## 4. Comparaison des performances

Le benchmark compare les deux solveurs sur les mêmes ensembles de puzzles.

### Métriques importantes

- **Temps total**: Performance brute de bout en bout
- **Temps moyen par puzzle**: Indicateur de la complexité moyenne
- **Taux de résolution**: Devrait être 100% pour les deux solveurs (ils sont complets)

### Ce qu'on observe généralement

| Aspect | OR-Tools CP-SAT | Z3 SMT |
|--------|-----------------|--------|
| **Puzzles faciles** | Très rapide | Rapide |
| **Puzzles difficiles** | Très rapide | Rapide |
| **Temps d'initialisation** | Négligeable | Légèrement plus long |
| **Overhead par puzzle** | Minimal | Création du modèle |

Les deux solveurs sont considérablement plus rapides que le backtracking simple, et comparables au backtracking avec MRV. Leur avantage principal est la **simplicité du code** et la **garantie de performance** même sur des problèmes plus complexes que le Sudoku standard.

In [ ]:
def benchmark_solvers(puzzles: List[str], name: str):
    """Compare les performances des solveurs."""
    print(f"\n{'='*60}")
    print(f"Benchmark: {name} ({len(puzzles)} puzzles)")
    print('='*60)
    
    solvers = {
        'OR-Tools CP-SAT': ORToolsSolver(),
        'Z3 SMT': Z3Solver()
    }
    
    results = {name: {'time': 0, 'solved': 0} for name in solvers}
    
    for puzzle_str in puzzles:
        for solver_name, solver in solvers.items():
            grid = SudokuGrid.from_string(puzzle_str)
            
            start = time.time()
            solved = solver.solve(grid)
            elapsed = time.time() - start
            
            results[solver_name]['time'] += elapsed
            if solved:
                results[solver_name]['solved'] += 1
    
    # Afficher résultats
    print(f"\n{'Solveur':<20} {'Résolus':<10} {'Temps total':<15} {'Temps moyen':<15}")
    print('-'*60)
    for solver_name, data in results.items():
        avg_time = (data['time'] / len(puzzles)) * 1000
        total_time = data['time'] * 1000
        print(f"{solver_name:<20} {data['solved']}/{len(puzzles):<8} {total_time:>10.2f} ms   {avg_time:>10.2f} ms")
    
    return results

# Benchmark
benchmark_solvers(easy_puzzles, "Puzzles Faciles")
benchmark_solvers(hard_puzzles, "Puzzles Difficiles (Top 11)")

## 5. Z3 Optimisé avec BitVectors

Les **BitVectors** sont une représentation alternative aux entiers symboliques dans Z3.

### Pourquoi utiliser des BitVectors?

Un entier symbolique (`Int`) dans Z3 représente un entier mathématique de précision arbitraire. Pour le Sudoku où les valeurs sont dans {1..9}, c'est surdimensionné.

Les BitVectors de 4 bits (`BitVec(name, 4)`) peuvent représenter les valeurs 0-15, suffisant pour {1..9}:

| Valeur | Binaire 4 bits |
|--------|----------------|
| 1 | 0001 |
| 9 | 1001 |
| 15 (max) | 1111 |

### Avantages des BitVectors

1. **Représentation fixe**: Le solveur connaît la taille exacte des valeurs
2. **Opérations bit-level**: Possibilité d'utiliser des techniques SAT pures
3. **Comparaisons plus simples**: `UGE` (unsigned >=) au lieu de `>=` arithmétique

### Limitations

- Pas de contrainte `Distinct` native pour BitVectors
- On doit utiliser des comparaisons par paires: O(n²) contraintes au lieu de O(n)
- L'overhead de création peut annuler le gain

### Note sur les performances

Dans la pratique, pour le Sudoku, la différence entre `Int` et `BitVec` est souvent marginale car Z3 optimise bien les deux. L'exercice est surtout pédagogique pour illustrer les différentes représentations disponibles.

In [ ]:
class Z3BitVectorSolver:
    """Solveur Z3 utilisant des BitVectors 4 bits."""
    
    def solve(self, grid: SudokuGrid) -> bool:
        """Résout avec BitVectors."""
        # BitVectors 4 bits (valeurs 1-9 tiennent dans 4 bits)
        X = [[BitVec(f'x_{i}_{j}', 4) for j in range(9)] for i in range(9)]
        
        solver = Solver()
        
        # Contraintes de domaine
        for i in range(9):
            for j in range(9):
                solver.add(UGE(X[i][j], 1))  # >= 1 (unsigned)
                solver.add(ULE(X[i][j], 9))  # <= 9 (unsigned)
        
        # Valeurs initiales
        for i in range(9):
            for j in range(9):
                if grid.cells[i][j] != 0:
                    solver.add(X[i][j] == grid.cells[i][j])
        
        # Contraintes Distinct (par paires pour BitVec)
        def add_all_different(cells):
            for i in range(len(cells)):
                for j in range(i + 1, len(cells)):
                    solver.add(cells[i] != cells[j])
        
        # Lignes
        for i in range(9):
            add_all_different([X[i][j] for j in range(9)])
        
        # Colonnes
        for j in range(9):
            add_all_different([X[i][j] for i in range(9)])
        
        # Blocs
        for box_row in range(3):
            for box_col in range(3):
                box_cells = [X[box_row * 3 + i][box_col * 3 + j] 
                             for i in range(3) for j in range(3)]
                add_all_different(box_cells)
        
        if solver.check() == sat:
            model = solver.model()
            for i in range(9):
                for j in range(9):
                    grid.cells[i][j] = model.evaluate(X[i][j]).as_long()
            return True
        
        return False

# Comparaison Z3 Int vs BitVec
print("=== Comparaison Z3 Int vs BitVector ===")

z3_int = Z3Solver()
z3_bv = Z3BitVectorSolver()

for i, puzzle_str in enumerate(hard_puzzles[:5]):
    print(f"\nPuzzle {i+1}:")
    
    grid1 = SudokuGrid.from_string(puzzle_str)
    start = time.time()
    z3_int.solve(grid1)
    t1 = (time.time() - start) * 1000
    
    grid2 = SudokuGrid.from_string(puzzle_str)
    start = time.time()
    z3_bv.solve(grid2)
    t2 = (time.time() - start) * 1000
    
    print(f"  Z3 Int:      {t1:>8.2f} ms")
    print(f"  Z3 BitVec:   {t2:>8.2f} ms")

## 6. Résumé et conclusions

### Comparaison des approches

| Solveur | Type | Avantages | Inconvénients |
|---------|------|-----------|---------------|
| **Backtracking** | Impératif | Simple, pédagogique | Lent sur puzzles difficiles |
| **Backtracking+MRV** | Impératif | Bon compromis | Heuristique manuelle |
| **OR-Tools CP-SAT** | Déclaratif | Très rapide, API claire | Dépendance externe |
| **Z3 Int** | Déclaratif | Flexible, expressif | Plus lent que CP-SAT |
| **Z3 BitVec** | Déclaratif | Optimisé bas niveau | Syntaxe plus complexe |

### Recommandations par cas d'usage

- **Apprentissage des algorithmes**: Commencer par le backtracking simple
- **Production / Performance**: OR-Tools CP-SAT
- **Recherche / Contraintes complexes**: Z3
- **Problèmes combinatoires variés**: OR-Tools (portfolio de solveurs)

### Au-delà du Sudoku

Ces solveurs peuvent résoudre une grande variété de problèmes:
- **OR-Tools**: Planification, routage, ordonnancement, bin packing
- **Z3**: Vérification de programmes, analyse de sécurité, synthèse de code

Le Sudoku est un excellent problème pédagogique car il est facile à comprendre mais assez complexe pour illustrer les concepts clés de la programmation par contraintes.

## Navigation

### Notebooks Python de cette serie
- [Sudoku-Python-Backtracking.ipynb](Sudoku-Python-Backtracking.ipynb) - Backtracking et MRV
- **Ce notebook**: OR-Tools CP-SAT et Z3 SMT
- [Sudoku-Python-Genetic.ipynb](Sudoku-Python-Genetic.ipynb) - Algorithmes genetiques (PyGAD)
- [Sudoku-Python-DancingLinks.ipynb](Sudoku-Python-DancingLinks.ipynb) - Algorithm X / Dancing Links

### Notebooks C# equivalents
- [Sudoku-3-ORTools.ipynb](Sudoku-3-ORTools.ipynb) - OR-Tools en C#
- [Sudoku-4-Z3.ipynb](Sudoku-4-Z3.ipynb) - Z3 en C#

---
**Navigation** : [<< Python Backtracking](Sudoku-Python-Backtracking.ipynb) | [Index](README.md) | [Python Genetic >>](Sudoku-Python-Genetic.ipynb)